Take combinations of ESC and anechoic data to be added as drone samples
Augment ESC data with pitch shifting

Potentially: augment full dataset with frequency masking
maybe add gaussian noise or white noise to some samples?

In [1]:
import librosa
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from glob import glob
import pandas as pd
from numpy import random
from scipy.io import wavfile

In [8]:
# escape = glob("C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\DADSONLY_NEW\*.wav")

# df = pd.DataFrame({
#     'Filename': escape
# })

# df.to_csv("../ESCAPEII.csv", index=False)

1. Create 10k new .wav files that consist of a random anechoic file and random ESC 50 file added together

In [2]:
anech = glob("C:\\Users\Alec\Desktop\dndMLApproach\\anechoic\*.wav")
ESC50 = glob("C:\\Users\Alec\Desktop\dndMLApproach\ESC50\\audio\*.wav")

In [216]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach\combined_augmented")
for i in range(10000):
    
    anech_sample = random.randint(len(anech))
    ESC50_sample = random.randint(len(ESC50))

    anech_wav, sr = librosa.load(anech[anech_sample], sr=44100)
    ESC50_wav, sr = librosa.load(ESC50[ESC50_sample], sr=44100)

    combined_wav = anech_wav + ESC50_wav
    
    wavfile.write("combined_aug" + "_" + str(i) + '.wav', sr, combined_wav)


2. Create 20k pitch shifted new .wav files

In [223]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach\pitch_shift_augmented")

for i in range(10000):
    sample = random.randint(len(anech))
    n_steps = (random.randint(11) + 1)
    y, sr = librosa.load(anech[sample], sr=44100)
    y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

    wavfile.write("anech_pitch_shifted_aug" + "_" + str(i) + '.wav', sr, y_shifted)

for i in range(10000):
    sample = random.randint(len(ESC50))
    n_steps = (random.randint(11) + 1)
    y, sr = librosa.load(ESC50[sample], sr=44100)
    y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

    wavfile.write("ESC50_pitch_shifted_aug" + "_" + str(i) + '.wav', sr, y_shifted)


3. Frequency mask on both anechoic and ESC50

In [351]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach\\freq_masked_augmented")

for i in range(10000):
    sample = random.randint(len(anech))
    random_list = [random.randint(50, 1025) for _ in range(10)]
    y, sr = librosa.load(anech[sample], sr=44100)
    S = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

    for r in random_list:
        S[r-50:r, :] = 0

    out = librosa.istft(S)

    wavfile.write("anech_freq_mask_aug" + "_" + str(i) + '.wav', sr, out)


for i in range(10000):
    sample = random.randint(len(ESC50))
    random_list = [random.randint(50, 1025) for _ in range(10)]
    y, sr = librosa.load(ESC50[sample], sr=44100)
    S = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

    for r in random_list:
        S[r-50:r, :] = 0

    out = librosa.istft(S)

    wavfile.write("ESC50_freq_mask_aug" + "_" + str(i) + '.wav', sr, out)

4. Generate CSV files for each augmented folder to make it easier for feature extraction

In [371]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach\combined_augmented")
comb = glob("*.wav")
comb = [os.path.join(os.getcwd(), file) for file in comb]
labels = []
for i in range(len(comb)):
    labels.append(0)

# Create a DataFrame
df = pd.DataFrame({
    'Filename': comb,
    'Label': labels
})

df.to_csv("../combined_augmented.csv", index=False)

In [372]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach\pitch_shift_augmented")
pitch = glob("*.wav")
pitch = [os.path.join(os.getcwd(), file) for file in pitch]
labels = []
for f in pitch:
    if f.startswith("anech"):
        labels.append(0)
    else:
        labels.append(1)

df = pd.DataFrame({
    'Filename': pitch,
    'Label': labels
})

df.to_csv("../pitch_shift_augmented.csv", index=False)

In [373]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach\\freq_masked_augmented")
freq = glob("*.wav")
freq = [os.path.join(os.getcwd(), file) for file in freq]
labels = []
for f in freq:
    if f.startswith("anech"):
        labels.append(0)
    else:
        labels.append(1)

df = pd.DataFrame({
    'Filename': freq,
    'Label': labels
})

df.to_csv("../freq_masked_augmented.csv", index=False)

In [3]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach\ESC50\\audio")
esc = glob("*.wav")
esc = [os.path.join(os.getcwd(), file) for file in esc]
labels = []
for f in esc:
    labels.append(1)

df = pd.DataFrame({
    'Filename': esc,
    'Label': labels
})

df.to_csv("../../esc50.csv", index=False)

In [4]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach\\anechoic")
an = glob("*.wav")
an = [os.path.join(os.getcwd(), file) for file in an]
labels = []
for f in an:
    labels.append(0)

df = pd.DataFrame({
    'Filename': an,
    'Label': labels
})

df.to_csv("../anechoic.csv", index=False)

5. Merge all of the CSVs to generate 1 csv to serve as the master for the training set

In [5]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach")
csvs = ['anechoic.csv', 'esc50.csv', 'freq_masked_augmented.csv', 'pitch_shift_augmented.csv', 'combined_augmented.csv']

dfs = [pd.read_csv(csv) for csv in csvs]  
merged_df = pd.concat(dfs, axis=0)  

sorted_df = merged_df.sort_values(by='Label')

sorted_df.to_csv('complete_dataset.csv', index=False)


In [380]:
t = "C:\\Users\Alec\Desktop\AnechoicDatasetWork\anechoic\DJIM600-FT-A-2426_20191126_chunk1.wav"

In [382]:
test = t.split("\\")

In [384]:
test[-1][:-4]

'DJIM600-FT-A-2426_20191126_chunk1'

In [21]:
os.chdir("C:\\Users\Alec\Desktop\dndMLApproach")

In [24]:
master = pd.read_csv("C:\\Users\Alec\Desktop\dndMLApproach\complete_dataset_features.csv")

In [27]:
def update_label_based_on_filename(filename):
    # Split the filename by '\'
    parts = filename.split('\\')
    # Check what the filename starts with and update the label accordingly
    if parts[-1][:-4].startswith('anech') or parts[-1][:-4].startswith('combined') or 'anech' in filename:
        return 0
    else:
        return 1

# # Apply the function to update the Label column
master['Label'] = master['Filename'].apply(update_label_based_on_filename)
anech = master[master['Label'] == 0]
esc = master[master['Label'] == 1]
print(anech.shape, esc.shape)

(39256, 3395) (30000, 3395)


In [28]:
# master.to_csv('complete_dataset_features.csv', index=False)

In [30]:
t = anech['Filename']
t.to_csv('anechFNames.csv', index = False)